In [1]:
import numpy as np
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
# from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
# from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
with open("/content/drive/MyDrive/CS598PSL_Project3/vocab.txt") as f:
  vocab = f.readlines()
vocab = [v[:-1].replace("_"," ") for v in vocab]

In [3]:
i = 1
folderName = '/content/drive/MyDrive/CS598PSL_Project3/split_' + str(i)
train_filename = folderName + '/' + 'train.tsv'
test_filename = folderName + '/' + 'test.tsv'
test_y_filename = folderName + '/' + 'test_y.tsv'

train_data = pd.read_csv(train_filename,sep='\t', header=0)
train_y = train_data['sentiment']
train_features = train_data.copy()
train_features = train_features.drop(['sentiment'],axis=1)

test_data = pd.read_csv(test_filename,sep='\t', header=0)
test_features = test_data['review']
    # print(test_data.head(5))
    # print(test_data.shape)
test_y_data = pd.read_csv(test_y_filename,sep='\t', header=0)

In [4]:
def clean_text(text):     
    return text.strip().lower()

In [5]:
vectorizer = CountVectorizer(ngram_range=(1, 4),vocabulary=vocab) 
#tfvectorizer = TfidfVectorizer(ngram_range=(1, 4))
classifier = LogisticRegression(penalty='l2', solver='liblinear',C=0.14)
#classifier = LogisticRegression(penalty='l1')
LRmodel = Pipeline([('vectorizer', vectorizer),
                      ('classifier', classifier)])

test_Y = test_y_data['sentiment']
LRmodel.fit(train_features['review'],train_y)
LRpred = LRmodel.predict_proba(test_features)
LRpred = LRpred[:,1]
result = LRmodel.predict(test_features)
print(f'Accuracy: {accuracy_score(test_Y,result)*100}%')
auc = roc_auc_score(test_Y, LRpred,average='micro')
print(f'AUC: {auc*100}%')
test = LRmodel['vectorizer'].get_feature_names_out()
new_vocab = test[LRmodel['classifier'].coef_[0]!=0]

Accuracy: 90.28%
AUC: 96.31678089557296%


In [13]:
np.savetxt("final_vocab.txt",new_vocab.astype(str),fmt='%s') 